In [10]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

import json
import pandas as pd
import numpy as np

from data.data_source import get_data_source
from data.treasury_curve import get_yield_curve, bump_curve, shocks
from models.pricing_models.bond_model import Bond

ds = get_data_source()


def run_valuation(asof_str):
    # 0) Parse / validate our date
    asof = pd.to_datetime(asof_str)
    if pd.isna(asof):
        raise ValueError(f"Could not parse date '{asof_str}'")

    # ── 1) Pull today’s inventory ────────────────────────────────────────────────
    inv_sql = f"""
    SELECT DISTINCT ON(cusip)
        cusip,
        int_rate,
        issue_date,
        maturity_date,
        price_per100,
        quantity,
        int_payment_frequency
    FROM tsy_inventory
    WHERE inventory_date = '{asof.date()}'
    ORDER BY cusip, inventory_date DESC;
    """
    inv = ds.query(inv_sql).to_pandas()
    if inv.empty:
        print(f"No inventory on {asof.date()}")
        return

    # ── 2) Load the base yield curve ───────────────────────────────────────────────
    base_yc = get_yield_curve(asof, ds)
    if base_yc is None:
        print(f"No curve for {asof.date()}")
        return

    # ── 3) Build our Bond objects ─────────────────────────────────────────────────
    bonds = [
        Bond(r.cusip,
             r.issue_date,
             r.maturity_date,
             r.int_rate,
             r.int_payment_frequency)
        for r in inv.itertuples()
    ]

    # ── 4) Price on the base curve ────────────────────────────────────────────────
    # returns: (dirty_PV, accrued_interest, clean_PV, dv01, krd_matrix)
    pvs_dirty, accrued_arr, pvs_clean, dv01s, krds_mat = (
        Bond.price_batch_with_sensitivities(bonds, asof, base_yc)
    )

    # ── 5) Copy inventory + populate base‐curve columns ────────────────────────────
    results = inv.copy().reset_index(drop=True)
    results['price_closedform']             = pvs_dirty
    results['clean_price_closedform']       = pvs_clean
    results['accrued_interest_closedform']  = accrued_arr
    results['dv01']                         = dv01s

    key_cols = ['krd1y','krd2y','krd3y','krd5y','krd7y','krd10y','krd20y','krd30y']
    for i, col in enumerate(key_cols):
        results[col] = krds_mat[:, i]

    # ── 6) Fetch that day’s PCA loadings (PC1, PC2, PC3) ──────────────────────────
    #    The JSONB “components” field comes back as bytes or a JSON string; we decode + load.
    pca_sql = f"""
    SELECT components
    FROM pca_results
    WHERE curve_type   = 'US Treasury Par'
      AND curve_date   = '{asof.date()}'
      AND n_components >= 3
    LIMIT 1;
    """
    pca_df = ds.query(pca_sql).to_pandas()
    if pca_df.empty:
        raise RuntimeError(f"No PCA results for curve_date={asof.date()} / curve_type='US Treasury Par'")

    raw = pca_df.loc[0, 'components']
    if isinstance(raw, (bytes, bytearray)):
        raw = raw.decode()
    components_list = json.loads(raw)
    components_np   = np.array(components_list, dtype=float)
    if components_np.shape[0] < 3:
        raise RuntimeError("PCA did not return at least 3 components on that date.")

    # Extract PC1, PC2, PC3 loadings (each ∈ ℝ^{n_tenors})
    pc1_loading = components_np[0]
    pc2_loading = components_np[1]
    pc3_loading = components_np[2]

    # ── 7) Define the tenor grid that PCA was run on ───────────────────────────────
    tenors = np.array([0.25, 0.5, 1.0, 2.0, 3.0, 5.0, 7.0, 10.0, 20.0, 30.0])
    if pc1_loading.shape[0] != tenors.shape[0]:
        raise RuntimeError("Mismatch between PCA‐component length and tenor grid.")

    # ── 8) Helper to build a PCA‐bumped yield curve ────────────────────────────────
    def make_pca_bumped_curve(base_yc, tenors, loading_vector, shift_bp):
        """
        Build a new yield‐curve function f(ttm_array) that = base_yc(ttm)
        + loading_vector * (shift_bp / 100), linearly interpolated across tenors.
        """
        base_rates   = base_yc(tenors)                                      # (n_tenors,)
        bumped_rates = base_rates + loading_vector * (shift_bp / 100.0)     # shift_bp in bps

        def f(ttm_arr):
            # Flatten + interp, then restore shape
            flat = np.interp(
                ttm_arr.ravel(),
                tenors,
                bumped_rates,
                left=bumped_rates[0],
                right=bumped_rates[-1]
            )
            return flat.reshape(ttm_arr.shape)

        return f

    # ── 9) Build columns for twelve PCA‐price bumps (±25bp, ±100bp, ±200bp on PC1/PC2/PC3)
    desired_pca_cols = {
        'price_closedform_pca1_u25bps':  (pc1_loading, +25),
        'price_closedform_pca1_d25bps':  (pc1_loading, -25),
        'price_closedform_pca2_u25bps':  (pc2_loading, +25),
        'price_closedform_pca2_d25bps':  (pc2_loading, -25),
        'price_closedform_pca3_u25bps':  (pc3_loading, +25),
        'price_closedform_pca3_d25bps':  (pc3_loading, -25),

        'price_closedform_pca1_u100bps': (pc1_loading, +100),
        'price_closedform_pca1_d100bps': (pc1_loading, -100),
        'price_closedform_pca2_u100bps': (pc2_loading, +100),
        'price_closedform_pca2_d100bps': (pc2_loading, -100),
        'price_closedform_pca3_u100bps': (pc3_loading, +100),
        'price_closedform_pca3_d100bps': (pc3_loading, -100),

        'price_closedform_pca1_u200bps': (pc1_loading, +200),
        'price_closedform_pca1_d200bps': (pc1_loading, -200),
        'price_closedform_pca2_u200bps': (pc2_loading, +200),
        'price_closedform_pca2_d200bps': (pc2_loading, -200),
        'price_closedform_pca3_u200bps': (pc3_loading, +200),
        'price_closedform_pca3_d200bps': (pc3_loading, -200),
    }

    # Initialize those PCA‐shock columns to zero first
    for col in desired_pca_cols:
        results[col] = 0.0

    # ── 10) Compute all twelve PCA bumps (dirty price) in one loop ───────────────
    for col_name, (loading_vec, shift_bp) in desired_pca_cols.items():
        yc_bumped = make_pca_bumped_curve(base_yc, tenors, loading_vec, shift_bp)
        pvs_dirty_bump, _, _, _, _ = Bond.price_batch_with_sensitivities(bonds, asof, yc_bumped)
        results[col_name] = pvs_dirty_bump

    # ── 11) Compute PCA‐directional DV01s (1 bp shift along PC1, PC2, PC3) ────────
    #      DV01 along PC_k = P_dirty(PC_k +1 bp) − P_dirty(base)
    yc_pca1_1bp = make_pca_bumped_curve(base_yc, tenors, pc1_loading, +1)
    yc_pca2_1bp = make_pca_bumped_curve(base_yc, tenors, pc2_loading, +1)
    yc_pca3_1bp = make_pca_bumped_curve(base_yc, tenors, pc3_loading, +1)

    pvs_pca1_1bp, _, _, _, _ = Bond.price_batch_with_sensitivities(bonds, asof, yc_pca1_1bp)
    pvs_pca2_1bp, _, _, _, _ = Bond.price_batch_with_sensitivities(bonds, asof, yc_pca2_1bp)
    pvs_pca3_1bp, _, _, _, _ = Bond.price_batch_with_sensitivities(bonds, asof, yc_pca3_1bp)

    results['pca1_dv01'] = pvs_pca1_1bp - results['price_closedform']
    results['pca2_dv01'] = pvs_pca2_1bp - results['price_closedform']
    results['pca3_dv01'] = pvs_pca3_1bp - results['price_closedform']

    # ── 12) Compute six parallel‐curve shocks (±25bp, ±100bp, ±200bp) ─────────────
    for label in shocks:
        results[f'price_closedform_{label}bps'] = 0.0

    for label, bp in shocks.items():
        yc_bumped = bump_curve(base_yc, bp)
        pvs_dirty_bump, _, _, _, _ = Bond.price_batch_with_sensitivities(bonds, asof, yc_bumped)
        results[f'price_closedform_{label}bps'] = pvs_dirty_bump

    # ── 13) Final housekeeping + drop near-maturity ───────────────────────────────
    results['valuation_date']   = asof.date()
    results['time_to_maturity'] = (
        pd.to_datetime(results['maturity_date']) - asof
    ).dt.days / 365.25
    results['coupon']           = results['int_rate'].fillna(0.0)

    alive_mask = results['time_to_maturity'] > 1e-4
    if not alive_mask.all():
        dropped = results.loc[~alive_mask, 'cusip'].tolist()
        print(f"⚠️  Dropping {len(dropped)} nearly-matured bonds: {dropped}")
    results = results.loc[alive_mask].reset_index(drop=True)

    # ── 14) Build single‐shot INSERT…ON CONFLICT upsert ────────────────────────────
    values = []
    for row in results.itertuples(index=False):
        vals = (
            f"('{row.cusip}',"                              # 1) cusip
            f"'{row.valuation_date}',"                       # 2) valuation_date
            f"{row.price_per100},"                           # 3) entry_price
            f"{row.coupon},"                                 # 4) coupon
            f"'{row.maturity_date}',"                        # 5) maturity_date
            f"{row.time_to_maturity},"                       # 6) time_to_maturity
            f"{row.dv01},"                                   # 7) dv01
            f"{row.krd1y},{row.krd2y},{row.krd3y},{row.krd5y},"   # 8–11) KRDs
            f"{row.krd7y},{row.krd10y},{row.krd20y},{row.krd30y}," # 12–15) KRDs
            f"{row.price_closedform},"                       # 16) base dirty price
            f"{row.price_closedform_u25bps},"                # 17) parallel +25 bp
            f"{row.price_closedform_d25bps},"                # 18) parallel −25 bp
            f"{row.price_closedform_u100bps},"               # 19) parallel +100 bp
            f"{row.price_closedform_d100bps},"               # 20) parallel −100 bp
            f"{row.price_closedform_u200bps},"               # 21) parallel +200 bp
            f"{row.price_closedform_d200bps},"               # 22) parallel −200 bp
            f"{row.price_closedform_pca1_u25bps},"           # 23) PCA1 +25 bp
            f"{row.price_closedform_pca1_d25bps},"           # 24) PCA1 −25 bp
            f"{row.price_closedform_pca2_u25bps},"           # 25) PCA2 +25 bp
            f"{row.price_closedform_pca3_u25bps},"           # 26) PCA3 +25 bp
            f"{row.price_closedform_pca1_u100bps},"          # 27) PCA1 +100 bp
            f"{row.price_closedform_pca1_d100bps},"          # 28) PCA1 −100 bp
            f"{row.price_closedform_pca2_u100bps},"          # 29) PCA2 +100 bp
            f"{row.price_closedform_pca3_u100bps},"          # 30) PCA3 +100 bp
            f"{row.price_closedform_pca1_u200bps},"          # 31) PCA1 +200 bp
            f"{row.price_closedform_pca1_d200bps},"          # 32) PCA1 −200 bp
            f"{row.price_closedform_pca2_u200bps},"          # 33) PCA2 +200 bp
            f"{row.price_closedform_pca3_u200bps},"          # 34) PCA3 +200 bp
            f"{row.price_closedform_pca3_d200bps},"          # 35) PCA3 −200 bp
            f"{row.pca1_dv01},"                              # 36) PCA1 DV01 (= P(PC1+1bp)−P(base))
            f"{row.pca2_dv01},"                              # 37) PCA2 DV01
            f"{row.pca3_dv01},"                              # 38) PCA3 DV01
            f"{row.quantity},"                               # 39) quantity
            f"{row.clean_price_closedform},"                 # 40) clean price
            f"{row.accrued_interest_closedform}"              # 41) accrued interest
            f")"
        )
        values.append(vals)

    vals_sql = ",".join(values)
    upsert_sql = f"""
    INSERT INTO tsy_valuations (
      cusip,
      valuation_date,
      entry_price,
      coupon,
      maturity_date,
      time_to_maturity,
      dv01,
      krd1y,
      krd2y,
      krd3y,
      krd5y,
      krd7y,
      krd10y,
      krd20y,
      krd30y,
      price_closedform,
      price_closedform_u25bps,
      price_closedform_d25bps,
      price_closedform_u100bps,
      price_closedform_d100bps,
      price_closedform_u200bps,
      price_closedform_d200bps,
      price_closedform_pca1_u25bps,
      price_closedform_pca1_d25bps,
      price_closedform_pca2_u25bps,
      price_closedform_pca3_u25bps,
      price_closedform_pca1_u100bps,
      price_closedform_pca1_d100bps,
      price_closedform_pca2_u100bps,
      price_closedform_pca3_u100bps,
      price_closedform_pca1_u200bps,
      price_closedform_pca1_d200bps,
      price_closedform_pca2_u200bps,
      price_closedform_pca3_u200bps,
      price_closedform_pca3_d200bps,
      pca1_dv01,
      pca2_dv01,
      pca3_dv01,
      quantity,
      clean_price_closedform,
      accrued_interest_closedform
    ) VALUES {vals_sql}
    ON CONFLICT(cusip, valuation_date) DO UPDATE SET
      entry_price                         = EXCLUDED.entry_price,
      coupon                              = EXCLUDED.coupon,
      maturity_date                       = EXCLUDED.maturity_date,
      time_to_maturity                    = EXCLUDED.time_to_maturity,
      dv01                                = EXCLUDED.dv01,
      krd1y                               = EXCLUDED.krd1y,
      krd2y                               = EXCLUDED.krd2y,
      krd3y                               = EXCLUDED.krd3y,
      krd5y                               = EXCLUDED.krd5y,
      krd7y                               = EXCLUDED.krd7y,
      krd10y                              = EXCLUDED.krd10y,
      krd20y                              = EXCLUDED.krd20y,
      krd30y                              = EXCLUDED.krd30y,
      price_closedform                    = EXCLUDED.price_closedform,
      price_closedform_u25bps             = EXCLUDED.price_closedform_u25bps,
      price_closedform_d25bps             = EXCLUDED.price_closedform_d25bps,
      price_closedform_u100bps            = EXCLUDED.price_closedform_u100bps,
      price_closedform_d100bps            = EXCLUDED.price_closedform_d100bps,
      price_closedform_u200bps            = EXCLUDED.price_closedform_u200bps,
      price_closedform_d200bps            = EXCLUDED.price_closedform_d200bps,
      price_closedform_pca1_u25bps        = EXCLUDED.price_closedform_pca1_u25bps,
      price_closedform_pca1_d25bps        = EXCLUDED.price_closedform_pca1_d25bps,
      price_closedform_pca2_u25bps        = EXCLUDED.price_closedform_pca2_u25bps,
      price_closedform_pca3_u25bps        = EXCLUDED.price_closedform_pca3_u25bps,
      price_closedform_pca1_u100bps       = EXCLUDED.price_closedform_pca1_u100bps,
      price_closedform_pca1_d100bps       = EXCLUDED.price_closedform_pca1_d100bps,
      price_closedform_pca2_u100bps       = EXCLUDED.price_closedform_pca2_u100bps,
      price_closedform_pca3_u100bps       = EXCLUDED.price_closedform_pca3_u100bps,
      price_closedform_pca1_u200bps       = EXCLUDED.price_closedform_pca1_u200bps,
      price_closedform_pca1_d200bps       = EXCLUDED.price_closedform_pca1_d200bps,
      price_closedform_pca2_u200bps       = EXCLUDED.price_closedform_pca2_u200bps,
      price_closedform_pca3_u200bps       = EXCLUDED.price_closedform_pca3_u200bps,
      price_closedform_pca3_d200bps       = EXCLUDED.price_closedform_pca3_d200bps,
      pca1_dv01                           = EXCLUDED.pca1_dv01,
      pca2_dv01                           = EXCLUDED.pca2_dv01,
      pca3_dv01                           = EXCLUDED.pca3_dv01,
      quantity                            = EXCLUDED.quantity,
      clean_price_closedform              = EXCLUDED.clean_price_closedform,
      accrued_interest_closedform         = EXCLUDED.accrued_interest_closedform,
      updated_at                          = CURRENT_TIMESTAMP;
    """
    ds.query(upsert_sql)
    print(f"✅ Valued {len(bonds)} bonds on {asof.date()} (with parallel‐ and PCA‐shocks, plus PCA‐DV01s).")


if __name__ == '__main__':
    arg = None
    if len(sys.argv) > 1 and not sys.argv[1].startswith('-'):
        arg = sys.argv[1]

    today = "2025-05-28"
    if arg is None:
        run_valuation(today)
    else:
        try:
            _ = pd.to_datetime(arg)
            run_valuation(arg)
        except:
            print(f"Invalid date '{arg}', defaulting to {today}")
            run_valuation(today)


getting data source for sandbox
✅ Valued 284 bonds on 2025-05-28 (with parallel‐ and PCA‐shocks, plus PCA‐DV01s).


In [ ]:
results